Importing Libraries

In [2]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from datetime import datetime 

Creating the Model

In [2]:
#creating a Convolutional Neural Network (CNN) using Keras
model_name = 'Face_trained_model_'+datetime.now().strftime("%H_%M_%S_") #A unique name for the model is created using the current time (hours, minutes, seconds).
model = Sequential(name = model_name)

model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(48, 48, 1)))#creates a cnn layer with a kernal size 3X3 and relu activation
model.add(BatchNormalization()) # normalizes the outputs of the previous layer, which helps improve training stability and speeds up convergence
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(BatchNormalization()) 
model.add(Conv2D(64, kernel_size=5, padding='same', activation='relu'))
model.add(BatchNormalization()) 
model.add(Dropout(0.2)) #randomly drops 20% of the units during training, which helps prevent overfitting
model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=5, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))#This down-samples the input by taking the maximum value over a 2x2 window, effectively reducing the spatial dimensions by half
model.add(Dropout(0.2))
model.add(Conv2D(256, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))


model.add(Flatten())#This layer converts the 2D feature maps from the cnn layers into a 1D vector, for further operations
model.add(Dense(256))#a fully connected layer with 256 units
model.add(BatchNormalization())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Dense(7, activation='softmax'))

model.summary()

C:\Users\nairs\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "Face_trained_model_11_11_01_"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 46, 46, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 46, 46, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 44, 44, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 44, 44, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 44, 44, 64)          │         102,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 44, 44, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 44, 44, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 42, 42, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 42, 42, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 40, 40, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 40, 40, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 40, 40, 128)         │         409,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 40, 40, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 20, 20, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 20, 20, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 18, 18, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 18, 18, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 9, 9, 256)           │              

 Total params: 6,413,703 (24.47 MB)

 Trainable params: 6,411,271 (24.46 MB)

 Non-trainable params: 2,432 (9.50 KB)

Compiling the Model

In [3]:
learning_rate = 0.001
optimizer = RMSprop(learning_rate=learning_rate)
model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
print("done")

done


Reading the Training Data

In [4]:
df = pd.read_csv("train_dataset.csv") 
id= df.iloc[:, 0].to_numpy() #converting the first column of df into a numpy array id

y=[]
pixels = df.iloc[:, 1].to_numpy()

for i in pixels:
    number_list = i.split()
    number_list = [int(num) for num in number_list]
    y.append(number_list)
y=np.array(y)

emotion=df.iloc[:, 2].to_numpy()
X = y.reshape(-1, 48, 48, 1) 
print(X.shape)

(5000, 48, 48, 1)


Training the Model

In [5]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=200,verbose=1,factor=0.2)#Reduces the learning rate when the validation loss plateaus
ch = ModelCheckpoint('models/'+model_name+'.keras', monitor='val_acc', verbose=0, save_best_only=True, mode='max')#Saves the model to disk only when the validation accuracy improves
es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=200)#Stops training when the validation loss stops improving for a given number of epochs (200 here, but we are anyways not going to do 200 epochs)


x_train, x_val, y_train, y_val = train_test_split(X, emotion, test_size=0.2, random_state=42)#splitting the data for validation to prevent overfitting
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=5, callbacks=[learning_rate_reduction, ch, es])
print("done")

Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 803s 6s/step - accuracy: 0.2688 - loss: 2.2292 - val_accuracy: 0.2420 - val_loss: 5.8156 - learning_rate: 0.0010
Epoch 2/5


C:\Users\nairs\AppData\Roaming\Python\Python312\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_acc available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 808s 6s/step - accuracy: 0.3604 - loss: 1.6369 - val_accuracy: 0.3650 - val_loss: 2.0370 - learning_rate: 0.0010
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 783s 6s/step - accuracy: 0.4450 - loss: 1.4511 - val_accuracy: 0.2980 - val_loss: 2.2613 - learning_rate: 0.0010
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 807s 6s/step - accuracy: 0.4791 - loss: 1.3539 - val_accuracy: 0.4290 - val_loss: 1.6260 - learning_rate: 0.0010
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 729s 6s/step - accuracy: 0.5079 - loss: 1.2560 - val_accuracy: 0.4560 - val_loss: 1.5264 - learning_rate: 0.0010
done


Predicting the Output

In [ ]:
dff = pd.read_csv("test_dataset.csv") 
idd= dff.iloc[:, 0].to_numpy() #converting the first column of df into a numpy array id

ss=[]
pixelss = dff.iloc[:, 1].to_numpy()
for i in pixelss:
    number_list = i.split()
    number_list = [int(num) for num in number_list]
    ss.append(number_list)
ss=np.array(ss)
y_pred = ss.reshape(-1, 48, 48, 1) 
y_pred = model.predict(y_pred)
y_pred = y_pred.argmax(axis=1)
#print(y_pred)

fdf=pd.DataFrame()
se = pd.Series(idd)
fdf['id'] = se.values

see=pd.Series(y_pred)
fdf['emotion']=see.values
fdf.to_csv('results.csv',index=False)#the result of this code is the submission which gave me the highest accuracy